In [18]:
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

### Extract CSVs into DataFrame

In [7]:
csv_file = "Resources/full_data.csv"
full_data_df = pd.read_csv(csv_file)
full_data_df.head(None)

,date,location,new_cases,new_deaths,total_cases,total_deaths,weekly_cases,weekly_deaths,biweekly_cases,biweekly_deaths
0,2020-02-24,Afghanistan,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,2020-02-25,Afghanistan,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
2,2020-02-26,Afghanistan,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,2020-02-27,Afghanistan,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,2020-02-28,Afghanistan,0.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
89957,2021-05-28,Zimbabwe,64.0,0.0,38918.0,1592.0,254.0,6.0,383.0,10.0
89958,2021-05-29,Zimbabwe,15.0,2.0,38933.0,1594.0,254.0,8.0,379.0,12.0
89959,2021-05-30,Zimbabwe,11.0,0.0,38944.0,1594.0,262.0,8.0,384.0,12.0
89960,2021-05-31,Zimbabwe,17.0,0.0,38961.0,1594.0,265.0,8.0,389.0,12.0


In [29]:
world_2021_file = "Resources/world-happiness-report-2021.csv"
world_2021_df = pd.read_csv(world_2021_file)
world_2021_df.head()
#Need to add year as column with rows indicating 2021

,Country name,Regional indicator,Ladder score,Standard error of ladder score,upperwhisker,lowerwhisker,Logged GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption,Ladder score in Dystopia,Explained by: Log GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Dystopia + residual
0,Finland,Western Europe,7.842,0.032,7.904,7.780,10.775,0.954,72.0,0.949,-0.098,0.186,2.43,1.446,1.106,0.741,0.691,0.124,0.481,3.253
1,Denmark,Western Europe,7.620,0.035,7.687,7.552,10.933,0.954,72.7,0.946,0.030,0.179,2.43,1.502,1.108,0.763,0.686,0.208,0.485,2.868
2,Switzerland,Western Europe,7.571,0.036,7.643,7.500,11.117,0.942,74.4,0.919,0.025,0.292,2.43,1.566,1.079,0.816,0.653,0.204,0.413,2.839
3,Iceland,Western Europe,7.554,0.059,7.670,7.438,10.878,0.983,73.0,0.955,0.160,0.673,2.43,1.482,1.172,0.772,0.698,0.293,0.170,2.967
4,Netherlands,Western Europe,7.464,0.027,7.518,7.410,10.932,0.942,72.4,0.913,0.175,0.338,2.43,1.501,1.079,0.753,0.647,0.302,0.384,2.798


### Create new data with select columns

In [28]:
new_full_data_df = full_data_df[['date','location','total_cases','total_deaths']]
new_full_data_df.loc[(new_full_data_df.location == 'United States'), ['date','location','total_cases','total_deaths']]
#new_full_data_df['year'] = new_full_data_df['date'].dt.year


,date,location,total_cases,total_deaths
85181,2020-01-22,United States,1.0,NaN
85182,2020-01-23,United States,1.0,NaN
85183,2020-01-24,United States,2.0,NaN
85184,2020-01-25,United States,2.0,NaN
85185,2020-01-26,United States,5.0,NaN
...,...,...,...,...
85673,2021-05-28,United States,33242999.0,593976.0
85674,2021-05-29,United States,33254998.0,594319.0
85675,2021-05-30,United States,33261731.0,594443.0
85676,2021-05-31,United States,33267507.0,594585.0


### Clean Dates

In [ ]:
new_full_data_df = new_full_data_df['total_deaths'].fillna(0)